In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup

import json
import time
import copy
from time import sleep
from selenium import webdriver

In [40]:
url = 'https://www.ebay.com/urw/Google-Home-Mini-Smart-Speaker-with-Google-Assistant-Chalk-GA00210-US-/product-reviews/239112716'
url1 = 'https://www.ebay.com/urw/Microsoft-Xbox-One-Day-One-Edition-500GB-Black-Console/product-reviews/175299667'
url2 = 'https://www.ebay.com/urw/Sony-PlayStation-4-500GB-Jet-Black-Console/product-reviews/167317351'
url3 = 'https://www.ebay.com/urw/Nintendo-Switch-HAC-001-01-32GB-Console-with-Gray-Joy-Con/product-reviews/25034355190?condition=new'

url4 = 'https://www.ebay.com/urw/Milwaukee-2407-20-12V-Cordless-Drill-Driver/product-reviews/28030363313'
url5 = 'https://www.ebay.com/urw/Makita-XPH12Z-18V-Lithium-Ion-Cordless-Hammer-Drill/product-reviews/809538545'
url6 = 'https://www.ebay.com/urw/DeWalt-DCD791B-20V-Max-XR-Li-Ion-Brushless-Compact-Drill-Tool-Only-/product-reviews/13031982751'

url7 = 'https://www.ebay.com/urw/Canon-EOS-5D-Mark-III-22-3MP-Digital-SLR-Camera-Black-Body-Only-/product-reviews/113297147'
url8 = 'https://www.ebay.com/urw/Nikon-D7500-20-9MP-Digital-SLR-Camera-Black-Body-Only-/product-reviews/237096362'
url9 = 'https://www.ebay.com/urw/Nikon-D750-24-3-MP-Digital-SLR-Camera-Black-Body-Only-/product-reviews/203954517'
r = requests.get(url1)
r.status_code

200

In [157]:
soup = BeautifulSoup(r.text,'html.parser')
#soup.find_all('p','review-item-content rvw-wrap-spaces')[0].find('span','show-full-review').text
#soup.find_all('div','review-section-header')
#lst = [i.text for i in soup.find_all('div','reviews-divider')]

category = []
product_type = []
product_name = []
avg_rating = []
price = []
review_text = []
star_rating = []
word_count = []

def parse_product_category(url):
    '''
    Returns the product category for a given product
    Input: URL of the product page to be parsed
    Output: String with the product category name
    '''
    
    return product_category
    
def parse_product_type(url):    
    '''
    Returns the product type for a given product
    Input: URL of the product page to be parsed
    Output: String with the product type name
    '''
    
    return product_type
    
#Store all reviews on a given page
for i in range(len(soup.find_all('p','review-item-content rvw-wrap-spaces'))):
    text = soup.find_all('p','review-item-content rvw-wrap-spaces')[i].find('span','show-full-review').text
    review_text.append(text)
    star_rating.append(float(soup.find_all('div','ebay-star-rating')[i+1].find('span','star-rating')["aria-label"].split(' ')[0]))
    avg_rating.append(float(soup.find_all('div','ebay-star-rating')[0].find('span','star-rating')["aria-label"].split(' ')[0]))
    product_name.append(soup.find('div','row').find('a','page-title')['title'])
    product_type.append('gaming console')
    category.append('gaming')
    price.append(float('500'))
    
df = pd.DataFrame({'category':category,'product_type':product_type,'product_name':product_name,'avg_rating':avg_rating,
                   'price':price,'review_text':review_text,'star_rating':star_rating})
df.head(10)


,category,product_type,product_name,avg_rating,price,review_text,star_rating
0,gaming,gaming console,Microsoft Xbox One Day One Edition 500GB Black...,4.5,500,you can easily download a PDF online product a...,5.0
1,gaming,gaming console,Microsoft Xbox One Day One Edition 500GB Black...,4.5,500,"ipe, aside from the two tone, was just how gro...",3.0
2,gaming,gaming console,Microsoft Xbox One Day One Edition 500GB Black...,4.5,500,(relative to the millions built) bad ones.In a...,4.0
3,gaming,gaming console,Microsoft Xbox One Day One Edition 500GB Black...,4.5,500,sed this review off of previous experience wit...,4.0
4,gaming,gaming console,Microsoft Xbox One Day One Edition 500GB Black...,4.5,500,"ps you the new console as soon as you call, be...",4.0
5,gaming,gaming console,Microsoft Xbox One Day One Edition 500GB Black...,4.5,500,lk about the good and bad things that I have n...,5.0
6,gaming,gaming console,Microsoft Xbox One Day One Edition 500GB Black...,4.5,500,", watch TV (Football usually) even while playi...",5.0
7,gaming,gaming console,Microsoft Xbox One Day One Edition 500GB Black...,4.5,500,ly easy to maneuver through all of its many as...,5.0
8,gaming,gaming console,Microsoft Xbox One Day One Edition 500GB Black...,4.5,500,"the Kinect 2.0. Other than that, it feels lik...",4.0
9,gaming,gaming console,Microsoft Xbox One Day One Edition 500GB Black...,4.5,500,g but interested to get one as sample because ...,5.0
